In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from Utils import exec_jupyter; exec_jupyter()

In [2]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)

from Utils import *

In [3]:
import sys
sys.path.insert(0, 'preprocess/')
import vectorizer
import pickle

In [4]:
vec = pickle.load(open('preprocess/MIMIC/vec_icd9.p', 'rb'))
vec.output_size = len(vec.label2idx)
add_frequencies(vec, vec.seqs['train']) 

In [5]:
import model.Attn_Word_Pert as M
Model = M.Model

In [6]:
diabetes_label = vec.label2idx['250.00']

In [7]:
X, Xt = vec.seqs['train'], vec.seqs['test']
y, yt = vec.label_one_hot['train'][:, diabetes_label], vec.label_one_hot['test'][:, diabetes_label]

In [8]:
len_t = np.argsort([len(x) for x in Xt])
Xt = [Xt[i] for i in len_t]
yt = [yt[i] for i in len_t]

In [9]:
pos_weight = 1 #len(y)/sum(y) - 1

In [10]:
from sklearn.metrics import classification_report, f1_score

def train(name='') :
    model = Model(vec.vocab_size, vec.word_dim, 32, dirname='diab', hidden_size=128, pre_embed=vec.embeddings, pos_weight=pos_weight)
    best_f1 = 0.0
    for i in tqdm_notebook(range(10)) :
        loss = model.train(X, y)
        o, he = model.evaluate(Xt)
        o = np.array(o)
        rep = classification_report(yt, (o > 0.5))
        f1 = f1_score(yt, (o > 0.5), pos_label=1)
        print(rep)
        stmt = '%s, %s' % (i, loss)
        if f1 > best_f1 :
            best_f1 = f1
            dirname = model.save_values(add_name=name, save_model=True)
            print("Model Saved", f1)
        else :
            dirname = model.save_values(add_name=name, save_model=False)
            print("Model not saved", f1)
        f = open(dirname + '/epoch.txt', 'a')
        f.write(stmt + '\n')
        f.write(rep + '\n')
        f.close()
    
    return model

In [11]:
# train(name='TEST_icd9')

# **EVALUATION**

In [12]:
def load_model(dirname) :
    model = Model(vec.vocab_size, vec.word_dim, 32, dirname='mimic', hidden_size=128, pre_embed=vec.embeddings)
    model.dirname = dirname
    model.load_values(dirname)
    return model

In [13]:
model = load_model('outputs/attn_word_diab/SunOct1412:56:302018_TEST_icd9/')

Setting Embedding


In [14]:
yt_hat, attn_hat = evaluate_and_print(model, Xt, yt)

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      1389
         1.0       0.79      0.81      0.80       340

   micro avg       0.92      0.92      0.92      1729
   macro avg       0.87      0.88      0.87      1729
weighted avg       0.92      0.92      0.92      1729



In [ ]:
plot_entropy(Xt, attn_hat)

# __SAMPLING__

In [ ]:
model.vec = vec
sampled_output = model.sampling_top(Xt, sample_vocab=100)

Non Zero :  tensor(185, device='cuda:0', dtype=torch.uint8)


In [ ]:
import pickle
pickle.dump(sampled_output, open(model.dirname + '/sampled.p', 'wb'))

In [ ]:
sampled_output = pickle.load(open(model.dirname + '/sampled.p', 'rb'))

In [ ]:
generate_medians_from_sampling_top(sampled_output, attn_hat)

In [ ]:
distractors = get_distractors(sampled_output, attn_hat)

**GRADIENTS**
=========

In [ ]:
grads = model.gradient_mem(Xt)
process_grads(grads)

In [ ]:
plot_grads(Xt, attn_hat, grads)

**Permutation**
===========

In [ ]:
perms = model.permute_attn(Xt)

In [ ]:
plot_permutations(perms, Xt, yt_hat, attn_hat)

**Adversarial**
===============

In [ ]:
adversarial_outputs = model.adversarial(Xt, _type='uniform')
ad_y, ad_attn = adversarial_outputs

In [ ]:
jds = plot_adversarial(Xt, yt_hat, attn_hat, adversarial_outputs)

In [ ]:
idx = list(np.where(np.logical_and(np.array(jds) > 0.3, yt_hat > 0.8))[0])[:30]
idx

In [ ]:
n = 731
print_adversarial_example(vec.map2words(X[n]), attn_hat[n], ad_attn[n])
print(yt_hat[n], ad_y[n])

**Copy Runs Previous and Subsequent**
=====================================

In [ ]:
copy_runs = model.copy_H_run(Xt)

In [ ]:
copy_outputs, copy_attn, copy_H_diff = copy_runs

In [ ]:
plot_attn_diff(Xt, attn_hat, copy_attn, "Attention vs Copy Attention", usehexbin=True)
plot_y_diff(X, attn_hat, yt_hat, copy_outputs, usehexbin=True)
plot_attn_diff(Xt, attn_hat, copy_H_diff, "Attention vs H difference", usehexbin=True)

**Zero Runs**
=============

In [ ]:
zero_runs = model.zero_H_run(Xt)

In [ ]:
zero_outputs, zero_H_diff = zero_runs

In [ ]:
plot_attn_diff(Xt, attn_hat, zero_H_diff, title="Attention vs zero H diff", usehexbin=True)
plot_y_diff(X, attn_hat, yt_hat, zero_outputs, usehexbin=True)

In [ ]:
remove_outputs = model.remove_and_run(Xt)